In [0]:
!pip install category_encoders 
!pip install apyori
!pip install mlxtend  
import tensorflow as tf
import pandas as pd 
import numpy as np 
import copy 
import category_encoders as ce 
from apyori import apriori 
import numpy as np
import re
import math
from itertools import chain 
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
#from mlxtend.frequent_patterns import apriori

####################JupyterNoteBook################

#df2 = pd.read_csv('C:/Users/Admin/Desktop/test.csv')

####################JupyterNoteBook################

#####################GOOGLE COLAB##################

from google.colab import files 

uploaded = files.upload()

import io 

df2 = pd.read_csv(io.BytesIO(uploaded['test_2.csv']))
#####################GOOGLE COLAB##################
database_copy= df2.copy()

database_copy['de35'].fillna(0,inplace=True) 
database_copy['de14'].fillna(0,inplace=True)

##################### Extracting City Name from "de_43"########################
newde_43 = []

for index in range(len(database_copy['de43'])):
    newde_43.append(database_copy['de43'][index].upper().replace(" ",""))

newde_43 = np.asarray(newde_43)
newColumn43 = []

listOfCities = ["MUMBAI","HYDERABAD","CHENNAI","BANGALORE","PUNE","THANE","SURAT","NOIDA","GURGAON","NEWDELHI","LUCKNOW","JAIPUR","AMAZON","OXIGEN"]
transactionType = ["ATM","AEP","IMP","POS"]

#Same Process as City Column for ATM,IMPS,AEP,POS

for i in newde_43:
    for j in listOfCities:
        m = re.search(j, i)
        if m:
            newColumn43.append(j)

newColumn43 = np.asarray(newColumn43)

dataset = pd.DataFrame({'City_Column':newColumn43})

database_copy['City_Column'] = dataset['City_Column']

##################### Extracting Transaction Type from "de_48"#################
newColumn48 = []
#arrayForde48 = np.asarray(database_copy['de48'])

for i in database_copy['de48']:
  
  for j in transactionType:
    m = re.search(j, i)
    if m:
        newColumn48.append(j)

newColumn48 = np.asarray(newColumn48)

dataset_transactionType = pd.DataFrame({'Transaction_Type':newColumn48})

database_copy['Transaction_Type'] = dataset_transactionType['Transaction_Type']      
    
newDatabase = database_copy.copy()

newDatabase.drop(columns=['de48', 'de43','de42','de52'])

sortedDatabaseWithPrice = newDatabase.sort_values(by=['de4'])

newdatabaseWithDroppedColumns = newDatabase.drop(columns=['de48', 'de43','de42','de52'])

priceSlab = [0,50000,100000,150000,200000]

col1 = []
col2 = []
col3 = []
col4 = []

#Dynamically Creating Variable names and assigning them lists#
g = globals()
for i in range(1, 5):
    g['depth_{0}'.format(i)] = []

#Dynamically Creating Variable names and assigning them lists#

categorizedPurcaseAmounts = []

for i in range(len(newdatabaseWithDroppedColumns['de4'])):
    eachTransactionAmount = int(math.ceil(newdatabaseWithDroppedColumns['de4'][i] / 1000.0)) * 1000
    categorizedPurcaseAmounts.append(eachTransactionAmount)
    
    
categorizedPurcaseAmounts= np.asarray(categorizedPurcaseAmounts)

dataset_purchaseAmounts = pd.DataFrame({'categorizedPurcaseAmounts':categorizedPurcaseAmounts})

database_copy['categorizedPurcaseAmounts'] = dataset_purchaseAmounts['categorizedPurcaseAmounts']      
    
#newDatabase_CategorizedPurchaseAmounts = database_copy.copy()
newDatabase= database_copy.copy()

#newDatabase_CategorizedPurchaseAmounts.drop(columns=['de4']) 
newDatabase.drop(columns=['de4'])  

#parsing the Column de_35 into Credit Card number, Expiry Date, Service Code, Last Digit########################

splitColumnde35 = []
splitColumnde35_Part1 = []
splitColumnde35_Part2 = []


for i in range(len(newdatabaseWithDroppedColumns['de35'])):
    if newdatabaseWithDroppedColumns['de35'][i] == 0:
        splitColumnde35.insert(i,str(newdatabaseWithDroppedColumns['de35'][i]))
   
    else:
        splitColumnde35.append(str(newdatabaseWithDroppedColumns['de35'][i]).split('=',1))
    
    
for i in range(len(splitColumnde35)):  
  
  if splitColumnde35[i]!=0:
    for j in range(len(splitColumnde35[i])):
        if j==0:
        
            tempvar = splitColumnde35[i][j]
            splitColumnde35_Part1.insert(i,tempvar)
            splitColumnde35_Part2.insert(i,tempvar)
        if j==1:
            tempvar = splitColumnde35[i][j]
            splitColumnde35_Part2.insert(i,tempvar)  
        
expirydates=[] 
servicecode=[] 
variable=[] 



split_list=[4,7]

for i in splitColumnde35_Part2:
    temp = zip(chain([0], split_list), chain(split_list, [None])) 
    for j,k in temp:        
        if j<4:      
            expirydates.append("".join(list(i[j : k])))
        elif 4<=j<7: 
            servicecode.append("".join(list(i[j : k])))         
        elif 7<=j<11:     
            variable.append("".join(list(i[j : k])))   
  
      
expirydates= np.asarray(expirydates)
newDatabase_expirydates = pd.DataFrame({'expirydates':expirydates})
newDatabase['expirydates'] = newDatabase_expirydates['expirydates']
newDatabase['expirydates'].fillna(0,inplace=True)


servicecode= np.asarray(servicecode)

for i in range(len(servicecode)):
    i = int(i)
    if not servicecode[i]:    
        servicecode[i]=0  
        servicecode[i] = int(servicecode[i])

newDatabase_servicecode = pd.DataFrame({'servicecode':servicecode})
newDatabase['servicecode'] = newDatabase_servicecode['servicecode']

  
discretionaryData= np.asarray(variable)

for i in range(len(discretionaryData)):
    if not discretionaryData[i]:    
        discretionaryData[i]=0  

newDatabase_discretionaryData = pd.DataFrame({'discretionaryData':discretionaryData})
newDatabase['discretionaryData'] = newDatabase_discretionaryData['discretionaryData']  

   
newDatabase_onehot = newDatabase.copy()
newDatabase_onehot = pd.get_dummies(newDatabase_onehot, columns=['de41','servicecode','expirydates','discretionaryData','de14','Transaction_Type','City_Column','categorizedPurcaseAmounts'], prefix = ['de41','servicecode','expirydates','discretionaryData','de14','Transaction_Type','City_Column','categorizedPurcaseAmounts'])

newDatabase_onehot = newDatabase_onehot.drop(columns=['de48', 'de43','de42','de52','de35']) #'de28' removed for Google colab

newDatabase_onehot_temp = newDatabase_onehot.filter(['de49','de0','de22','de32','de2','de3','City_Column_MUMBAI'], axis=1)

records=[]    

records = newDatabase_onehot.values.tolist()


dataset = [[1, 'Beer', 'Rice', 'Chicken'],
           ['1', 'Beer', 'Rice'],
           ['1', 'Beer'],
           ['1', 'Bananas'],
           ['Milk', 'Beer', 'Rice', 'Chicken'],
           ['Milk', 'Beer', 'Rice'],
           ['Milk', 'Beer'],
           ['1', 'Bananas']]
temp=[]
columnNames=[]
for col in newDatabase_onehot:
    columnNames.append(col)
#print(columnNames)
count = 0
recordsArray = np.asarray(records)

#print('original records')
#print(records[0:2]) 

for i in records[0:2]:
  rowIndex = records.index(i)
  for j in i:
    columnIndex = i.index(j)
    if records[rowIndex][columnIndex]== 1:
      records[rowIndex][columnIndex] = columnNames[columnIndex]

#print('modified records')
#print(records[0:2]) 
         
        
dataset = np.asarray(records[0:2])

strdataset=[]
for i in dataset:
    count = 0
    temp_list=[]
    for j in i:
        if type(j)!=str and count<=len(i):
            j = str(j)
            count = count + 1
            temp_list.append(j)    
        elif count<=len(i):
            temp_list.append(j) 
            count = count+1
    strdataset.append(temp_list)  

te = TransactionEncoder()
te_ary = te.fit(strdataset).transform(strdataset)
#print(te.columns_)

df = pd.DataFrame(te_ary, columns=te.columns_)
apriori(df, min_support=0.6, use_colnames=True)
 
#print(apriori(df, min_support=0.6, use_colnames=True))

   

Saving test_2.csv to test_2 (13).csv


,support,itemsets
0,1.0,(0)
1,1.0,(30)
2,1.0,(356)
3,1.0,(de14_0.0)
4,1.0,(servicecode_201)
5,1.0,"(30, 0)"
6,1.0,"(356, 0)"
7,1.0,"(de14_0.0, 0)"
8,1.0,"(servicecode_201, 0)"
9,1.0,"(356, 30)"
